In [1]:
import pandas as pd
import seaborn as sns
import plotly.plotly as py
from scipy import integrate
from io import StringIO
from os import walk
from os.path import join

# .out files will be stored in separate directories in the master directory
# Program should be able to plot all files from a single directory on the same graph

# Directory where all the .out files are stored
directory = r'../../SANS_experiments/Total_GU/Primus/bottom_top/june2_usermeetinganalysis'

# Get a list of all of the files in the directory
f = []
for (dirpath, dirnames, filenames) in walk(directory):
    f.extend(filenames)
    break

# Only keep the .out files
outlist = [x for x in f if x[-3:].upper() == 'OUT']

# Create blank master dataframe
all_df = pd.DataFrame()

# This is the text that precedes the data in every file
start = 'Distance distribution  function of particle  \n\n\n      R          P(R)      ERROR\n\n  '

# Column names
headers = ['R', 'P(R)', 'ERROR']

# Loop through each .out file
for outfile in outlist:
    
    # Get the data from the file
    x = open(join(directory, outfile), 'r')
    
    # Get the text from the file
    text = x.read()
    
    # Get the location of the data within the text
    idx = text.find(start)
    
    # Remove unnecessary text and put into StringIO object for reading into a table
    testdata = StringIO(text[idx+len(start):])
    
    # Read the data into a table
    df = pd.read_csv(testdata, names=headers, sep="  ", engine='python')
    
    # Calculate the area under the curve
    auc = integrate.trapz(y = df['P(R)'], x = df['R'])
    
    # Create new column that is divided by the area under the curve
    df['P(R)_Norm'] = df['P(R)']/auc
    
    # Create label column and fill it with the file name
    df['File'] = outfile
    
    # Add data to master dataframe
    all_df = all_df.append(df, ignore_index=False)
    
    # Close the file
    x.close()    

# Parameters for Plotly graph
fig = {
    'data': [
        {
            'x': all_df[all_df['File']==file]['R'],
            'y': all_df[all_df['File']==file]['P(R)_Norm'],
            'name': file, 'mode': 'markers',
        } for file in list(all_df['File'].unique())
    ],
    'layout': {
        'xaxis': {'title': 'R'},
        'yaxis': {'title': "P(R)_Norm"}
    }
}
    
# Create Plotly graph
url = py.plot(fig, filename='Cfit V_hRib_42D Vacant')


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
import plotly
plotly.tools.set_credentials_file(username='EmilyDoris', api_key='Onj0fKwX3oqu30Jw9gSf')